# PyJive workshop: Time-dependent analysis
    
In this notebook, usage of the `ExplicitTimeModule` and `NewmarkModule` for time-dependent analysis of structures is explored. This is for solving dynamics problems in the time domain. The first of these is for explicit analysis with the central difference scheme, the second for implicit analysis with Newmark time integration. 
    
There is also an `NLNewmarkModule` for implicit analysis of nonlinear problems, but we will not use it here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import contextlib
import os
from urllib.request import urlretrieve

import sys
pyjivepath = '../../../pyjive/'
sys.path.append(pyjivepath)

if not os.path.isfile(pyjivepath + 'utils/proputils.py'):
    print('\n\n**pyjive cannot be found, adapt "pyjivepath" above or move notebook to appropriate folder**\n\n')
    raise Exception('pyjive not found')

import main
from utils import proputils as pu
from names import GlobNames as gn

%load_ext autoreload
%autoreload 2

%matplotlib widget

In [ ]:
# define a function we will use for plotting force and displacement signals
def plotForceDisp(globdat):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(globdat['lodi']['left']['load']['dx'])
    plt.plot(globdat['lodi']['right']['load']['dx'])
    plt.legend(['left','right'])
    plt.ylabel('F')
    plt.subplot(1,2,2)
    plt.plot(globdat['lodi']['left']['disp']['dx'])
    plt.plot(globdat['lodi']['right']['disp']['dx'])
    plt.legend(['left','right'])
    plt.ylabel('u')
    plt.show()

def plotDispVeloAccel(globdat):
    plt.figure()
    plt.plot(globdat['acc']['right']['disp']['dx'])
    plt.plot(globdat['acc']['right']['velo']['dx'])
    plt.plot(globdat['acc']['right']['accel']['dx'])
    plt.legend(['disp','velo','accel'])
    plt.ylabel('a')
    plt.show()

# download input files (if necessary)
def findfile(fname):
    url = "https://gitlab.tudelft.nl/cm/public/drive/-/raw/main/transient/" + fname + "?inline=false"
    if not os.path.isfile(fname):
        print(f"Downloading {fname}...")
        urlretrieve(url, fname)

findfile("bar_explicit.pro")
findfile("bar_implicit.pro")
findfile("bar_harmonic.pro")
findfile("beam.geom")
    

### Analysis 1: Wave propagation in a bar, explicit
The first case concerns wave propagation in a bar. It is a bar problem, we have three models that can solve bar problems, the `BarModel`, the `SolidModel`, and the `FrameModel`. We use the last of these for its dedicated postprocessing routines. 

We first run the case with the explicit time module. Run the analysis below and have a look at force and displacement signals. 

In [ ]:
props = pu.parse_file('bar_explicit.pro')
globdat = main.jive(props)


In [ ]:
plotForceDisp(globdat)

### Analysis 2: Wave propagation in a bar, implicit

Now the same case is analyzed with the NewmarkModule. Check again what happens when you change the time step size. 

What happens when you add numerical damping? 


In [ ]:
props = pu.parse_file('bar_implicit.pro')
globdat = main.jive(props)


In [ ]:
plotForceDisp(globdat)

### Analysis 3: Harmonically loaded bar
Now, instead of a constant load we apply an harmonic load. Check the input file how this is done.

In [ ]:
props = pu.parse_file('bar_harmonic.pro')
globdat = main.jive(props)

In [ ]:
plotDispVeloAccel(globdat)

What happens if you make the frequency of the harmonic load much lower? Try with a 100 times, don't forget to adapt deltaTime at two places in the properties.

In [ ]:
props['model']['neum']['deltaTime'] = 1;
props['stepper']['deltaTime'] = 1;
props['model']['neum']['timeSignal']  = 'np.sin(0.01*t)**2'
globdat = main.jive(props)

In [ ]:
plotForceDisp(globdat)

Can you also get this result with the `ExplicitTimeModule`?

In [ ]:
props = pu.parse_file('bigM.pro')

globdat = main.jive(props)